In [0]:
filepath = "/FileStore/tables/bronze_schema/netflix_titles.csv"

In [0]:
df = (spark.read.format("csv").option("inferSchema", True).option("header", True)
            .load(filepath))
df.createOrReplaceTempView("df_netflix_titles")

In [0]:
display(df)

In [0]:
jsonFilePath = "/FileStore/tables/bronze_schema/nested_people_1000.json"

In [0]:
df_json = (spark.read.format("json").option("multiline", True)
.option("header", True)
.option("inferSchema", True)
.load(jsonFilePath))
display(df_json)

In [0]:
from pyspark.sql.functions import col

ddfExpandedJson = df_json.select(
    "email",
    "error",
    "id",
    "name",
    col("address.city").alias("address_city"),
    col("address.number").alias("address_number"),
    col("address.street").alias("address_street"),
    col("address.zip").alias("address_zip"),
    col("profile.active").alias("profile_active"),
    col("profile.age").alias("profile_age"),
    col("profile.join_date").alias("profile_join_date")
)




In [0]:
display(ddfExpandedJson)

In [0]:
df_json.select("*", "address.*", "profile.*").drop(col("address"), col("profile")).display()

In [0]:
ddfExpandedJson.filter((col("name") == 'Hannah') & (col("address_city") == 'New York')).display()

In [0]:
ddfExpandedJson.filter(((col("address_city") == 'Seattle') |(col("address_city") == 'Chicago')) &(col("name") == 'Alice')).display()

In [0]:
ddfExpandedJson.filter((col("address_city").isin('Seattle', 'Seattle', 'Chicago') ) &(col("name") == 'Alice')).display()

In [0]:
ddfExpandedJson.columns

In [0]:
ddfExpandedJson = ddfExpandedJson.withColumnRenamed("profile_join_date", "memberJoinDate")

In [0]:
ddfExpandedJson.columns

In [0]:
ddfExpandedJson.withColumn("idExponent", col("id")*2).display()

In [0]:
ddfExpandedJson.withColumn("regerx", regexp_replace(col("email"),  "@.*", "@gmail.com")).display()

In [0]:
ddfExpandedJson.withColumn("castedzip", col("address_zip").cast("double")).display()

In [0]:
ddfExpandedJson.sort(col("address_number").desc()).display()

In [0]:
ddfExpandedJson.sort(["address_number", "address_zip"], ascending=[False,True]).display()

In [0]:
ddfExpandedJson.limit(2).display()

In [0]:
ddfExpandedJson.drop(col("error"), col("memberJoinDate")).display()

In [0]:
ddfExpandedJson.dropDuplicates().display()

In [0]:
ddfExpandedJson.drop_duplicates(subset = ['email', 'name']).display()

In [0]:
ddfExpandedJson.dropna(subset=['email']).display()

In [0]:
ddfExpandedJson.distinct().display()

In [0]:
from pyspark.sql import Row

# First DataFrame
ddf1 = spark.createDataFrame([
    Row(id=1, name="Alice", city="New York"),
    Row(id=2, name="Bob", city="Chicago")
])

# Second DataFrame (note: column order different, and extra column)
ddf2 = spark.createDataFrame([
    Row(name="Charlie", city="Seattle", id=3),
    Row(name="Diana", city="Boston", id=4)
])


In [0]:
ddf1.unionByName(ddf2).display()